In [142]:
import cairosvg
from SquareEnv2 import space2MultiPolygon
import numpy as np
import math
from shapely.geometry.polygon import orient
from PIL import Image
import io
import shapely as shp
from shapely.affinity import rotate
from numpy import deg2rad
import re

A = .000000000000001
scale = 30

In [99]:
def shapely2png(shapely, img_size, dx=margin/2, dy=margin/2):
    svg = shapely.svg(fill_color='#A3E0C2')
    svg = f'<svg xmlns="http://www.w3.org/2000/svg" width="{img_size+margin}" height="{img_size+margin}"><g transform="scale(1, -1) translate({dx}, -{img_size+dy})">{svg}</g></svg>'
    return Image.open(io.BytesIO(cairosvg.svg2png(bytestring=svg, background_color='#e7e7e7', output_height=img_size, output_width=img_size)))

In [145]:
def shapely2png(shapely_geom, img_size, margin=10):
    # Calculate the bounding box of the Shapely geometry
    minx, miny, maxx, maxy = shapely_geom.bounds

    # Create the SVG with the appropriate viewBox and transform attributes
    svg = shapely_geom.svg(fill_color='#A3E0C2')
    viewBox_width = maxx - minx
    viewBox_height = maxy - miny
    viewBox = f"{minx-margin} {miny-margin} {viewBox_width+margin} {viewBox_height+margin}"

    svg = f'''<svg xmlns="http://www.w3.org/2000/svg"
                width="{img_size+margin*2}" height="{img_size+margin*2}"
                viewBox="{viewBox}"
                preserveAspectRatio="xMidYMid meet">
                <g transform="scale(1, -1) translate({-margin/2}, {-maxy - miny + margin/2})">
                    {svg}
                </g>
               </svg>'''

    return Image.open(io.BytesIO(cairosvg.svg2png(bytestring=svg,
                                                  background_color='#e7e7e7',
                                                  output_height=img_size,
                                                  output_width=img_size)))


In [105]:
def angle_between_points(a, b):
    # Convert tuples to numpy arrays
    a = np.array(a)
    b = np.array(b)

    # Calculate the dot product
    dot_product = np.dot(a, b)

    # Calculate the magnitudes of the vectors
    magnitude1 = np.linalg.norm(a)
    magnitude2 = np.linalg.norm(b)

    # Calculate the cosine of the angle
    cos_angle = dot_product / (magnitude1 * magnitude2)

    # Calculate the angle in radians
    angle_radians = np.arccos(cos_angle)

    # Convert the angle to degrees (optional)
    return np.degrees(angle_radians), angle_radians

In [149]:
def parse_state(state, side_len=True, svg=False, image=True, dx=0, dy=0, size=None, rnd=3):
    if type(state) is str:
        try:
            state = eval(re.sub(r'(?<!\[)\s+', ',', state))
        except Exception as err:
            raise Exception(f'Failed to parse\n`{state}`') from err
    data = np.array(state).reshape((int(len(state)/3),3))
    x, y = space2MultiPolygon(data).minimum_rotated_rectangle.exterior.coords.xy
    coords = list(space2MultiPolygon(data).minimum_rotated_rectangle.exterior.coords)
    dx = coords[1][0] - coords[0][0]
    dy = coords[1][1] - coords[0][1]
    angle = np.degrees(np.arctan2(dy, dx))
    edge_length = (shp.Point(x[0], y[0]).distance(shp.Point(x[1], y[1])), shp.Point(x[1], y[1]).distance(shp.Point(x[2], y[2])))
    _side_len = max(edge_length)

    if side_len:
        print(f'Side Length:', round(_side_len, rnd))

    rtn = data.copy()
    if svg:
        display(rotate(space2MultiPolygon(data), -angle))
    if image:
        data[:,0:2] *= scale
        display(shapely2png(rotate(space2MultiPolygon(data, side_len=scale), -angle), (_side_len if size is None else size) * scale))

    return rtn
# s = np.array([2.236551  , 3.9033625 , 1.4088103 , 2.3927202 , 1.7010379 ,
#        0.26339227, 4.988667  , 2.819215  , 0.44801196, 4.0960484 ,
#        2.3633945 , 0.44870156, 1.8259761 , 2.7990499 , 0.23920502,
#        4.5029454 , 3.740701  , 0.50919217, 2.9186528 , 2.8260698 ,
#        0.03003966, 1.0648408 , 4.161766  , 0.44944984, 3.282405  ,
#        3.9102738 , 1.4019105 , 2.8136287 , 5.0225973 , 0.4494782 ,
#        3.7264938 , 5.432935  , 0.4494319 ])
# s2 = [1.6045002 , 4.261224  , 1.364817  , 2.6066422 , 4.003253  ,
#        1.4243941 , 0.9940915 , 1.389026  , 1.3809589 , 2.0872374 ,
#        2.2017298 , 0.07752558, 3.8678687 , 0.7987075 , 1.3654404 ,
#        3.3107805 , 1.867319  , 0.4832412 , 1.4298679 , 3.269493  ,
#        1.3778586 , 4.3748217 , 3.661359  , 1.3693848 , 3.0315716 ,
#        2.8833907 , 0.4313028 , 2.3423557 , 1.1547815 , 1.5115314 ,
#        4.259763  , 2.6476803 , 1.3913232 ]
# gap = .708+A
# parse_state([
#     .5, .5, 0,
#     1.5+gap+A, .5, 0,
#     .5, 1.5+A+gap, 0,
#     1.5+A+gap, 1.5+A+gap, 0,
#     1+A+(gap/2), 1+A+(gap/2), (math.pi / 4),
# ])
# parse_state(s)
# print('-'*20)
# parse_state(s2)
# pass

In [42]:
def display_df(df, *args, **kwargs):
    for row in df.iter_rows():
        print(f'Episode {row[0]}:')
        # if 'image' not in kwargs:
        #     kwargs['image'] = True
        parse_state(row[4], *args, **kwargs)